In [ ]:
import os
import sys
workplace_root = r'H:\My Code\nlp\nlp_course_project-master\news_cls_recm'
sys.path.append(workplace_root)
sys.path.append(os.path.join(workplace_root, 'dev'))

# 读入数据并预处理及Token化处理

实测NLTK加Jieba分词与Tokenize的速度太慢了，后续还是继续使用BERT，在分类模型读取过程中直接使用，不反复读入

In [ ]:

# ///////
import json
import jieba
import random
import torch
import argparse
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import sys
sys.path.insert(0, r'H:\My Code\nlp\nlp_course_project-master\news_cls_recm\dev')  # 添加当前目录到搜索路径的最前面

from tqdm import tqdm
from wordcloud import WordCloud, STOPWORDS
from collections import defaultdict
from matplotlib.font_manager import FontProperties

from utils import read_and_sample_single_folder
from utils import CustomDataset

from transformers import BertTokenizer, BertModel
# ///
bert_path = r'H:\My Code\nlp\nlp_course_project-master\news_cls_recm\pretrained_weights\bert-base-chinese'
bert_tokenizer = BertTokenizer.from_pretrained(bert_path)
bert_model = BertModel.from_pretrained(bert_path)
"""
    parser = argparse.ArgumentParser(description='Build Up the Model and Train/Test')
    parser.add_argument('--data_path', type=str, default='data', help='Path to the data')
    parser.add_argument('--mode', type=str, default='train', help='Mode')
    parser.add_argument('--bert', type=str, default='pretrained_weights/bert-base-chinese', help='Path to the data')
    parser.add_argument('--max_length', type=int, default=512, help='Maximum length of the sequence')
    parser.add_argument('--num_classes', type=int, default=14, help='Number of classes')
    parser.add_argument('--batch_size', type=int, default=8, help='Batch size')
    parser.add_argument('--num_epochs', type=int, default=100, help='Number of epochs')
    parser.add_argument('--lr', type=float, default=0.00001, help='Learning rate')
    parser.add_argument('--weight_decay', type=float, default=0.01, help='Weight decay')
    parser.add_argument('--input_size', type=int, default=768, help='Input size')
    parser.add_argument('--hidden_size', type=int, default=256, help='Hidden size')
    parser.add_argument('--num_layers', type=int, default=8, help='Number of layers')
    parser.add_argument('--num_heads', type=int, default=8, help='Number of heads')
    parser.add_argument('--dropout_rate', type=float, default=0.1, help='Dropout rate')
    parser.add_argument('--bidirectional', action='store_true', help='Whether to use bidirectional LSTM')    
    parser.add_argument('--save_folder', type=str, default='weights', help='Path to save the model')
    parser.add_argument('--model_name', type=str, default='bert_naive', help='Name of the model')
    parser.add_argument('--device', type=str, default='cuda', help='Device to use')
    parser.add_argument('--sample_rate', type=int, default=0.01, help='Number of samples to use')
    parser.add_argument('--seed', type=int, default=114514, help='Seed for reproducibility')
    parser.add_argument('--continue_training', type=str, default='weights/bert_naive_20240501161942.pth', help='Continue training from a checkpoint')
    return parser.parse_args()
"""
args = argparse.Namespace(
    max_length=512, 
    num_classes=14, 
    batch_size=8, 
    num_epochs=100, 
    lr=0.00001, 
    weight_decay=0.01, 
    input_size=768, 
    hidden_size=256, 
    num_layers=8, 
    num_heads=8, 
    dropout_rate=0.1, 
    bidirectional=True, 
)

In [ ]:
# __author__ = 'Katherine'
# from matplotlib.font_manager import FontManager
# import subprocess

# fm = FontManager()
# mat_fonts = set(f.name for f in fm.ttflist)

# output = subprocess.check_output(
#     'fc-list :lang=zh -f "%{family}\n"', shell=True)
# output = output.decode('utf-8')
# print '*' * 10, '系统可用的中文字体', '*' * 10
# print output
# zh_fonts = set(f.split(',', 1)[0] for f in output.split('\n'))
# available = mat_fonts & zh_fonts

# print('*' * 10, '可用的字体', '*' * 10)
# for f in available:
#     print(f)


In [8]:
#TODO 把这个colormap、label_dict、chinese_font都放到Kernel中
font = '/usr/share/fonts/MyFonts/simhei.ttf'
chinese_font = FontProperties(fname=font)

try:
    with open(os.path.join( '../class_indices.json'), 'r') as f:
    # with open(os.path.join(workplace_root, 'dev/class_indices.json'), 'r') as f:
        class_indices = {v: k for k,v in json.load(f).items()}
    print(class_indices)
except Exception as e:
    print("An error occurred:", e)
label_dict = {v: k for k, v in class_indices.items()}
print(label_dict)

NameError: name 'FontProperties' is not defined

In [ ]:
data_path = r'H:\大二\冬季\创新实训2\THUCNews\THUCNews'
# data_path = os.path.join(workplace_root, 'THUCNews')
random.seed(114514)
data_path, data_label = list(read_and_sample_single_folder(data_path, sample_rate=0.01)) # 先采样0.001的数据进行后续测试

## 词云绘制

In [ ]:
data_cls = defaultdict(list)
for path, label in tqdm(zip(data_path, data_label)):
    with open(path, 'r', encoding='utf-8') as f:
        data = f.readlines()
    temp = ''
    for line in data:
        temp += line
    data_cls[label].append(temp)
print(data_cls)
tgt_cls = '体育'
wc = WordCloud(
    background_color='white',   #背景设置为白色
    font_path=font,             #字体
    max_words=200,              #最大显示的关键词数量
    stopwords=STOPWORDS,        #使用上面导入停用词表
    max_font_size=250,          #最大字体
    random_state=30,            #设置随机状态数，及配色的方案数
    height=860,                 #如果使用默认图片，则可以设置高
    margin=2,                   #图片属性
    width=1000,                 #设置宽
).generate(''.join(data_cls[label_dict[tgt_cls]]))


plt.figure(figsize = (20,20), dpi=300)
plt.imshow(wc , interpolation = 'bilinear')
plt.axis('off')
plt.title(f'{tgt_cls}词云图',fontsize=100, font=chinese_font)
plt.show()

In [ ]:
# 初始化 defaultdict 用于存储长度计数
data_length = defaultdict(lambda: defaultdict(int))

# 遍历每个数据文件并统计文本长度
for label, data in data_cls.items():
    for text in data:
        data_length[label][len(text)] += 1  # 更新对应label和长度的计数器

tgt_dict = data_length[label_dict[tgt_cls]]
tgt_df = pd.DataFrame(tgt_dict.keys(), index=tgt_dict.values())
fig, ax = plt.subplots(1, 1, figsize=(20, 10))

sns.histplot(tgt_df, ax=ax, color='teal', kde=True)
sns.kdeplot(tgt_df, color='black', ax=ax) 
plt.title(f'{tgt_cls} 文本长度分布', font=chinese_font)
plt.xlabel('文本长度', font=chinese_font)

# 文本推荐

In [ ]:
rec_map = {
    bert_model.embeddings(bert_tokenizer(            
        text=class_indices[k],
        max_length=512,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )['input_ids']): k for k in data_cls.keys()
}

In [ ]:
input_ = "篮球比赛"
input_embedding = bert_model.embeddings(bert_tokenizer(            
    text=input_,
    max_length=512,
    padding='max_length',
    truncation=True,
    return_tensors='pt'
)['input_ids'])

similarities = {}
for recommend_vector, cls in rec_map.items():
    similarity = torch.nn.functional.cosine_similarity(recommend_vector[0], input_embedding[0])
    similarities[cls] = torch.mean(similarity)

sorted_recommendations = sorted(similarities.items(), key=lambda x: x[1].item(), reverse=True)

top_recommendation = sorted_recommendations[0]

print("最相似的推荐内容:", class_indices[top_recommendation[0]])
print("相似度:", top_recommendation[1].item())
print("推荐内容:\n", *data_cls[top_recommendation[0]][:10])

# 文本聚类

In [ ]:
data_cls.keys()

In [ ]:
print(*data_cls[1][:2])

In [ ]:
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE

In [ ]:
text_vectors = []
for cls in tqdm(data_cls.keys()):
    texts = data_cls[cls]  
    cls_vectors = np.array([np.mean(bert_model.embeddings(bert_tokenizer(            
        text=text,
        max_length=512,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )['input_ids']).squeeze(0).detach().numpy(), axis=0) for text in texts])
    text_vectors.extend(cls_vectors)

In [ ]:
plt.rcParams['font.sans-serif']=['SimHei']###解决中文乱码
plt.rcParams['axes.unicode_minus']=False


text_vectors = np.array(text_vectors)
# 现在可以使用K-means进行聚类
num_clusters = len(data_cls)  # 假设聚成5类
kmeans = KMeans(n_clusters=num_clusters, random_state=0)
cluster_labels = kmeans.fit_predict(text_vectors)

# 使用t-SNE进行降维可视化
tsne = TSNE(n_components=2, random_state=0)
text_tsne = tsne.fit_transform(text_vectors)

In [ ]:
color_map = [
    'salmon', 'burlywood', 'darkseagreen', 'lightseagreen', 'coral',
    'steelblue', 'teal','darkviolet', 'crimson', 'tomato', 'orangered', 'slategray', 'slateblue', 'darkorange', 'gold', 'yellowgreen',
    'olivedrab', 'darkgoldenrod', 'sienna', 'peru', 'chocolate', 'saddlebrown',
    'firebrick', 'maroon' , 'lightsalmon',
    'lightcoral', 'indianred', 'rosybrown', 'sandybrown'
]

# 绘制聚类可视化结果
plt.figure(figsize=(10, 8))
for i in range(num_clusters//2):
    cluster_points = text_tsne[cluster_labels == i]
    plt.scatter(cluster_points[:, 0], cluster_points[:, 1], label=f'{class_indices[i]}')
plt.title('Text Clustering Visualization')
plt.legend()
plt.show()

# 绘制聚类可视化结果
plt.figure(figsize=(10, 8))
for i in range(num_clusters//2, num_clusters):
    cluster_points = text_tsne[cluster_labels == i]
    plt.scatter(cluster_points[:, 0], cluster_points[:, 1], label=f'{class_indices[i]}')
plt.title('Text Clustering Visualization')
plt.legend()
plt.show()

# 绘制聚类可视化结果
plt.figure(figsize=(10, 8))
for i in range(num_clusters):
    cluster_points = text_tsne[cluster_labels == i]
    plt.scatter(cluster_points[:, 0], cluster_points[:, 1], label=f'{class_indices[i]}', c=color_map[i])
plt.title('Text Clustering Visualization')
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import silhouette_score, adjusted_rand_score

# 计算轮廓系数
silhouette_avg = silhouette_score(text_vectors, cluster_labels)

print(f"The average silhouette_score is : {silhouette_avg}")

# 文本分类

In [ ]:
import sys
sys.path.insert(0, r'H:\My Code\nlp\nlp_course_project-master\news_cls_recm')

from model import Bert_Naive

基本功能测试,及实际使用可直接利用的代码

In [ ]:
text = """
那英蛰伏九年十月将发行新专辑 蔡健雅任制作人
　　新浪娱乐讯 9年未出唱片的那英(微博)将于10月份发行全新个人专辑《那又怎样》。新专辑由蔡健雅(微博)担任制作人，收录了包括蔡健雅、吴青峰、阿弟仔、小寒等两岸三地以及新加坡、马来西亚的音乐人创作歌曲。下周，那英将赴台进行新专辑封面与音乐录像带的拍摄工作。
　　据悉，那英对新专辑《那又怎样》从发想、制作、企划概念都全程亲自参与和执行，整张专辑将呈现那英九年来对生活整理与体验的心得。专辑制作人蔡健雅自言那英是自己的偶像，正是那英的那首《白天不懂夜的黑》开启了她华语音乐的启蒙之门，因此当初那英打电话邀请她担任专辑制作人时，蔡健雅还一度因为担心自己会做不好而想推辞，因为要为自己的偶像制作专辑是个极大的荣幸也是极度的压力，但是那英对蔡健雅充满信心，一直告诉蔡健雅“你可以的，就是你了”。
　　此外，专辑的合作团队，那英选择了“亚神音乐”，因为这里有一群曾经与她长期合作且信任的伙伴，那英在台湾的事务也都将交由“亚神音乐”处理。
"""

net = Bert_Naive(bert=bert_model, args=args)
net.load_state_dict(torch.load('/home/drew/Desktop/nlp_course_project/news_cls_recm/weights/bert_naive_20240501174421.pth'))

In [ ]:
tokens = bert_tokenizer(text)
input_ids, attention_mask = torch.tensor(tokens['input_ids']).unsqueeze(0), torch.tensor(tokens['attention_mask']).unsqueeze(0)
print('文本:', text)
print('类别:', class_indices[torch.argmax(torch.nn.functional.softmax(net(input_ids, attention_mask), dim=-1)).item()])

# 命名实体识别

In [7]:
import hanlp
HanLP = hanlp.load(hanlp.pretrained.mtl.CLOSE_TOK_POS_NER_SRL_DEP_SDP_CON_ELECTRA_SMALL_ZH) 
word=HanLP([text])

100% 114.3 MiB 547.4 KiB/s ETA:  0 s [=========================================]
Decompressing C:\Users\15517\AppData\Roaming\hanlp\mtl/close_tok_pos_ner_srl_dep_sdp_con_electra_small_20210111_124159.zip to C:\Users\15517\AppData\Roaming\hanlp\mtl
100%  41.2 KiB  41.2 KiB/s ETA:  0 s [=========================================]
Decompressing C:\Users\15517\AppData\Roaming\hanlp\transformers/electra_zh_small_20210706_125427.zip to C:\Users\15517\AppData\Roaming\hanlp\transformers
100%  19.4 KiB  19.4 KiB/s ETA:  0 s [=========================================]
Decompressing C:\Users\15517\AppData\Roaming\hanlp\thirdparty\file.hankcs.com\corpus/char_table.json.zip to C:\Users\15517\AppData\Roaming\hanlp\thirdparty\file.hankcs.com\corpus


NameError: name 'text' is not defined

In [ ]:
word.pretty_print()